This code uses the Levenshtein distance metric to classify Mayo Clinic vs Homeopathe International documents

In [1]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from numpy import random
from Levenshtein import distance, hamming
import sys
import pickle
from sklearn.cross_validation import train_test_split, KFold
from nltk.util import ngrams

###Read in documents

In [2]:
#Read in Mayo documents
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)
print type(Mayo_list)

35
<type 'list'>


In [3]:
#Read in Homeopathic documents
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(str(stuff.decode("ascii", "ignore")))
    k += 1
    if k == 35:  #match the number of Mayo Clinic articles
        break
print len(Homeo_list)
print type(Homeo_list)

35
<type 'list'>


###Build corpora

In [4]:
#Build a training corpus for each source
Mayo_train_corpus = ''
Homeo_train_corpus = ''

i = 0
while i < len(Mayo_list):
    Mayo_train_corpus = Mayo_train_corpus + Mayo_list[i]
    i += 1

i = 0
while i < len(Homeo_list):
    Homeo_train_corpus = Homeo_train_corpus + Homeo_list[i]
    i += 1

In [22]:
#Build a training corpus of both sources together
total_corpus = ''
i = 0
while i < len(Mayo_list):
    total_corpus = total_corpus + Mayo_list[i]
    i += 1
i = 0
while i < len(Homeo_list):
    total_corpus = total_corpus + Homeo_list[i]
    i += 1

In [13]:
%%time

#distances of documents from each source

#Provide report on comparison of training documents
MayoT_MayoC = []
print 'Mayo test to Mayo training corpus distance:'
for i in range(35):
    dist = distance(Mayo_train_corpus, Mayo_list[i])
    MayoT_MayoC.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

HomeoT_HomeoC = []
print 'Homeo test to Homeo training corpus distance:'
for i in range(35):
    dist = distance(Homeo_train_corpus, Homeo_list[i])
    HomeoT_HomeoC.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

HomeoT_MayoC = []
print 'Homeo test to Mayo training corpus distance:'
for i in range(35):
    dist = distance(Mayo_train_corpus, Homeo_list[i])
    HomeoT_MayoC.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

MayoT_HomeoC = []
print 'Mayo test to Homeo training corpus distance:'
for i in range(35):
    dist = distance(Homeo_train_corpus, Mayo_list[i])
    MayoT_HomeoC.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)


Mayo test to Mayo training corpus distance:
Test Distance 0: 35918
Test Distance 1: 36998
Test Distance 2: 36477
Test Distance 3: 35954
Test Distance 4: 36033
Test Distance 5: 36433
Test Distance 6: 36165
Test Distance 7: 36868
Test Distance 8: 36775
Test Distance 9: 36175
Test Distance 10: 36900
Test Distance 11: 36375
Test Distance 12: 36362
Test Distance 13: 36787
Test Distance 14: 36631
Test Distance 15: 36645
Test Distance 16: 36953
Test Distance 17: 36714
Test Distance 18: 36824
Test Distance 19: 36522
Test Distance 20: 36817
Test Distance 21: 36770
Test Distance 22: 36557
Test Distance 23: 36117
Test Distance 24: 36581
Test Distance 25: 36147
Test Distance 26: 36330
Test Distance 27: 36159
Test Distance 28: 35710
Test Distance 29: 36540
Test Distance 30: 36672
Test Distance 31: 36906
Test Distance 32: 36979
Test Distance 33: 36489
Test Distance 34: 36743

Homeo test to Homeo training corpus distance:
Test Distance 0: 717227
Test Distance 1: 743984
Test Distance 2: 749659
Test Di

In [23]:
%%time

#distances of documents from total corpus
Mayo_total = []
print 'Mayo test to total corpus distance:'
for i in range(35):
    dist = distance(total_corpus, Mayo_list[i])
    Mayo_total.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

Homeo_Total = []
print 'Homeo test to total corpus distance:'
for i in range(35):
    dist = distance(total_corpus, Homeo_list[i])
    Homeo_Total.append(dist)
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

Mayo test to total corpus distance:
Test Distance 0: 791528
Test Distance 1: 792608
Test Distance 2: 792087
Test Distance 3: 791564
Test Distance 4: 791643
Test Distance 5: 792043
Test Distance 6: 791775
Test Distance 7: 792478
Test Distance 8: 792385
Test Distance 9: 791785
Test Distance 10: 792510
Test Distance 11: 791985
Test Distance 12: 791972
Test Distance 13: 792397
Test Distance 14: 792241
Test Distance 15: 792255
Test Distance 16: 792563
Test Distance 17: 792324
Test Distance 18: 792434
Test Distance 19: 792132
Test Distance 20: 792427
Test Distance 21: 792380
Test Distance 22: 792167
Test Distance 23: 791727
Test Distance 24: 792191
Test Distance 25: 791757
Test Distance 26: 791940
Test Distance 27: 791769
Test Distance 28: 791320
Test Distance 29: 792150
Test Distance 30: 792282
Test Distance 31: 792516
Test Distance 32: 792589
Test Distance 33: 792099
Test Distance 34: 792353

Homeo test to total corpus distance:
Test Distance 0: 754816
Test Distance 1: 781573
Test Distance

In [16]:
int(np.mean(MayoT_MayoC))

36515

In [17]:
int(np.mean(HomeoT_HomeoC))

734021

In [18]:
int(np.mean(HomeoT_MayoC))

31490

In [19]:
int(np.mean(MayoT_HomeoC))

754536

In [33]:
#total corpus comparisons
Mayo_total_mean = int(np.mean(Mayo_total))

In [34]:
Homeo_total_mean = int(np.mean(Homeo_Total))

In [ ]:
'''
strategy: make seperate mayo and homeo training corpora
find distance of each document in that corpus to the corpus
average these distances
test document should be closer to that average distance for proper classification
'''

###Train/test splitting

In [26]:
#Test/train split on documents
Mayo_array = np.array(Mayo_list)
Mayo_labels = np.array(['Mayo' for x in range(len(Mayo_list))])
x_train_Mayo, x_test_Mayo, y_train_Mayo, y_test_Mayo = train_test_split(Mayo_array,
                                Mayo_labels, test_size=0.25, random_state=42)

Homeo_array = np.array(Homeo_list)
Homeo_labels = np.array(['Homeo' for x in range(len(Homeo_list))])
x_train_Homeo, x_test_Homeo, y_train_Homeo, y_test_Homeo = train_test_split(Homeo_array,
                                Homeo_labels, test_size=0.25, random_state=42)

print len(x_train_Homeo), len(x_test_Homeo), len(y_train_Homeo), len(y_test_Homeo)
x_test = np.concatenate((x_test_Mayo, x_test_Homeo), axis=1)
y_test = np.concatenate((y_test_Mayo, y_test_Homeo), axis=1)
y_test

26 9 26 9


array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo',
       'Mayo', 'Homeo', 'Homeo', 'Homeo', 'Homeo', 'Homeo', 'Homeo',
       'Homeo', 'Homeo', 'Homeo'], 
      dtype='|S5')

In [98]:
#Cross validation split on documents

Mayo_array = np.array(Mayo_list)
Mayo_labels = np.array(['Mayo' for x in range(len(Mayo_list))])
x_train_Mayo = []
y_train_Mayo = []
x_test_Mayo = []
y_test_Mayo = []

kf = KFold(35, n_folds=7)
for train_index, test_index in kf:
    #x_train_Mayo = Mayo_array[train_index]
    #x_test_Mayo = Mayo_array[test_index]
    #y_train_Mayo = Mayo_labels[train_index]
    #y_test_Mayo = Mayo_labels[test_index]
    x_train_Mayo.append(Mayo_array[train_index])
    y_train_Mayo.append(Mayo_labels[train_index])
    x_test_Mayo.append(Mayo_array[test_index])
    y_test_Mayo.append(Mayo_labels[test_index])
    
Homeo_array = np.array(Homeo_list)
Homeo_labels = np.array(['Homeo' for x in range(len(Homeo_list))])
x_train_Homeo = []
y_train_Homeo = []
x_test_Homeo = []
y_test_Homeo = []

kf = KFold(35, n_folds=7)
for train_index, test_index in kf:
    #x_train_Homeo = Mayo_array[train_index]
    #x_test_Homeo = Mayo_array[test_index]
    #y_train_Homeo = Mayo_labels[train_index]
    #y_test_Homeo = Mayo_labels[test_index]
    x_train_Homeo.append(Homeo_array[train_index])
    y_train_Homeo.append(Homeo_labels[train_index])
    x_test_Homeo.append(Homeo_array[test_index])
    y_test_Homeo.append(Homeo_labels[test_index])
    
print len(x_train_Mayo[0]), len(y_train_Mayo)
print len(x_train_Homeo), len(y_train_Homeo)
print len(x_test_Mayo[0]), len(y_test_Mayo)
#x_test = x_test_Mayo + x_test_Homeo
#y_test = y_test_Mayo + y_test_Homeo
#x_test = zip(x_test_Mayo, x_test_Homeo)
#y_test = zip(y_test_Mayo, y_test_Homeo)
x_test = list(np.concatenate((x_test_Mayo, x_test_Homeo), axis=1))
y_test = list(np.concatenate((y_test_Mayo, y_test_Homeo), axis=1))
#print x_test_Mayo, y_test_Mayo
#print len(x_test[0])
print len(x_test[0])
#the_dist = distance(str(x_train[0][0]), str(x_test[0][1]))
#print the_dist

#result_list = doc_classifier(Mayo_train_corpus, Homeo_train_corpus, x_train_Mayo,
#                             x_train_Homeo, y_train_Mayo, y_train_Homeo,
#                             x_test, y_test)

30 7
7 7
5 7
10


In [97]:
print y_test
y_test = list(y_test)
print type(y_test)
'''
for q in range(len(x_test_Mayo)):
    x_test.append(np.concatenate(x_test_Mayo[q], x_test_Homeo[q]))
    y_test.append(y_test_Mayo[q])
    y_test.append(y_test_Homeo[q])
print type(y_test)
y_test[0]
'''

[['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']
 ['Mayo' 'Mayo' 'Mayo' 'Mayo' 'Mayo' 'Homeo' 'Homeo' 'Homeo' 'Homeo'
  'Homeo']]
<type 'list'>


'\nfor q in range(len(x_test_Mayo)):\n    x_test.append(np.concatenate(x_test_Mayo[q], x_test_Homeo[q]))\n    y_test.append(y_test_Mayo[q])\n    y_test.append(y_test_Homeo[q])\nprint type(y_test)\ny_test[0]\n'

###(Make training averages manually)

In [6]:
#Generate lists of average of each document's distance to the corpus
#Takes a while, unpickle if you need the results again
Mayo_dist_list = []
for i in range(len(x_train_Mayo)):
    dist = distance(x_train_Mayo[i], Mayo_train_corpus)
    Mayo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i)) 

Homeo_dist_list = []
for i in range(len(x_train_Homeo)):
    dist = distance(x_train_Homeo[i], Homeo_train_corpus)
    Homeo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i))

print Mayo_dist_list
print Homeo_dist_list

27[36645, 36979, 36165, 35918, 36175, 36433, 36147, 36714, 36522, 36906, 36540, 35954, 36770, 36477, 36672, 36868, 36787, 36489, 36033, 36900, 36557, 36775, 35710, 36159, 36330, 36743, 36581, 36824]
[741312, 698054, 711261, 717227, 737927, 704019, 729752, 710556, 748280, 749779, 738161, 748691, 695484, 749659, 739214, 702475, 745263, 746409, 715791, 741390, 745688, 728466, 744161, 745175, 752936, 737117, 742878, 743057]


In [143]:
#Mayo_mean = np.mean(Mayo_dist_list, dtype=int)
#Homeo_mean = np.mean(Homeo_dist_list, dtype=int)
print Mayo_mean
print Homeo_mean

36489
65110


###Classification function

In [99]:
#Classify remaining documents to see if it is close to appropriate average
def doc_classifier(corpus1, x_train_1, x_train_2, y_train_1, y_train_2,
                   x_test, y_test, dist1_mean=0, dist2_mean=0):
    assert type(corpus1), type(corpus2) == 'str'
    assert type(x_train_1), type(x_train_2) == 'list'
    assert type(y_train_1), type(y_train_2) == 'list'
    assert type(x_test), type(y_test) == 'list'
    assert len(x_train_1) == len(x_train_2)
    
    #Find average of distances of x_train_1 to corpus1
    if dist1_mean == 0 and dist2_mean == 0:
        for i in range(len(x_train_1)):
            dist_1 = distance(corpus1, x_train_1[i])
            dist1_mean = dist1_mean + dist_1
            #sys.stdout.write("\r" + '' + str(i))
            #dist_2 = distance(corpus2, x_train_2[i])
            #dist2_mean = dist1_mean + dist_2
            sys.stdout.write("\r" + 'Average ' + str(i + 1) + ' of ' +
                             str(len(x_train_1)) + ' completed.')
        dist1_mean = dist1_mean / len(x_train_1)
        dist2_mean = dist2_mean / len(x_train_2)
    
    result_list = [['Known', 'Predicted']]
    for i in range(len(x_test)):
        #print y_test[i]
        test_dist_1 = distance(corpus1, x_test[i])
        #test_dist_2 = distance(corpus2, x_test[i])
        if abs(test_dist_1 - dist1_mean) < abs(test_dist_1 - dist2_mean):
            result_list.append([y_test[i], 'From Corpus 1'])
        else:
            result_list.append([y_test[i], 'From Corpus 2'])
        sys.stdout.write("\r" + 'Test ' + str(i + 1) + ' of ' + str(len(x_test))
                         + ' completed.')
    
    return result_list  #, dist1_mean, dist2_mean
    
    '''    
        if abs(Mayo_dist - Mayo_mean) < abs(Homeo_dist - Homeo_mean):
            print 'Mayo Clinic source'
        else:
            print 'Homeopathe Intl source'
        print 'Test Distance ' + str(i) + ': ' + str(Mayo_dist)

    print ''
    '''

###Results

In [36]:
%%time

#Single test/train split run
sing_result_list = []
sing_result_list = doc_classifier(total_corpus, x_train_Mayo,
                             x_train_Homeo, y_train_Mayo, y_train_Homeo,
                             x_test, y_test, Mayo_total_mean, Homeo_total_mean)
#sing_result_list.append(result_list)

Mayo
Test 1 of 18 completed.Mayo
Test 2 of 18 completed.Mayo
Test 3 of 18 completed.Mayo
Test 4 of 18 completed.Mayo
Test 5 of 18 completed.Mayo
Test 6 of 18 completed.Mayo
Test 7 of 18 completed.Mayo
Test 8 of 18 completed.Mayo
Test 9 of 18 completed.Homeo
Test 10 of 18 completed.Homeo
Test 11 of 18 completed.Homeo
Test 12 of 18 completed.Homeo
Test 13 of 18 completed.Homeo
Test 14 of 18 completed.Homeo
Test 15 of 18 completed.Homeo
Test 16 of 18 completed.Homeo
Test 17 of 18 completed.Homeo
Test 18 of 18 completed.CPU times: user 7min, sys: 2.69 s, total: 7min 3s
Wall time: 7min 8s


In [37]:
sing_result_list

[['Known', 'Predicted'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 2'],
 ['Homeo', 'From Corpus 2'],
 ['Homeo', 'From Corpus 2'],
 ['Homeo', 'From Corpus 2'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 2']]

In [102]:
%%time

#Cross validated run
cross_result_list = []
for i in range(len(x_train_Mayo)):
    cross_result_list.append(doc_classifier(total_corpus, x_train_Mayo[i],
                             x_train_Homeo[i], y_train_Mayo[i], y_train_Homeo[i],
                             x_test[i], y_test[i], Mayo_total_mean, Homeo_total_mean))
    sys.stdout.write("\r" + ' ' + str(i))


 6CPU times: user 32min 14s, sys: 4.83 s, total: 32min 19s
Wall time: 32min 21s


In [103]:
cross_result_list

[[['Known', 'Predicted'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 2']],
 [['Known', 'Predicted'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 2']],
 [['Known', 'Predicted'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Mayo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 2'],
  ['Homeo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 1'],
  ['Homeo', 'From Corpus 2']],
 [['Known', 'Predicted'],
  ['Mayo

In [115]:
successes = 0
total = 0
Mayo_successes = 0
Homeo_successes = 0

for a in cross_result_list:
    for b in a:
        if b[0] != 'Known':
            total += 1
        if b[0] == 'Mayo' and b[1] == 'From Corpus 1':
            successes += 1
            Mayo_successes += 1
        if b[0] == 'Homeo' and b[1] == 'From Corpus 2':
            successes += 1
            Homeo_successes += 1
print 'Total accuracy: ' + str(float(successes) / total)
print 'Mayo accuracy: ' + str(float(Mayo_successes) / (total * 0.5))
print 'Homeo accuracy: ' + str(float(Homeo_successes) / (total * 0.5))

Total accuracy: 0.828571428571
Mayo accuracy: 1.0
Homeo accuracy: 0.657142857143


In [112]:
bigrams = ngrams(y_test, 3)
#print bigrams[0]
for gram in bigrams:
    print gram
    

(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo'

###Vectorizer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(Mayo_list)
x = vectorizer.transform(Mayo_list)
#'humans' in vectorizer.get_feature_names()
print x

In [ ]:
x_array = x.toarray()
df = pd.DataFrame(x_array, columns=vectorizer.get_feature_names())

###Pickling

In [20]:
#with open('Mayo_dist_list.pkl', 'w') as picklefile:
#    pickle.dump(Mayo_dist_list, picklefile)
#with open('corpus_averages/Mayo_mean.pkl', 'w') as picklefile:
#    pickle.dump(Mayo_mean, picklefile)
#with open('corpus_averages/Homeo_mean.pkl', 'w') as picklefile:
#    pickle.dump(Homeo_mean, picklefile)
with open('corpus_averages/MayoT_MayoC.pkl', 'w') as picklefile:
    pickle.dump(MayoT_MayoC, picklefile)
with open('corpus_averages/HomeoT_HomeoC.pkl', 'w') as picklefile:
    pickle.dump(HomeoT_HomeoC, picklefile)
with open('corpus_averages/HomeoT_MayoC.pkl', 'w') as picklefile:
    pickle.dump(HomeoT_MayoC, picklefile)
with open('corpus_averages/MayoT_HomeoC.pkl', 'w') as picklefile:
    pickle.dump(MayoT_HomeoC, picklefile)

In [112]:
with open('Homeoo_dist_list.pkl', 'w') as picklefile:
    pickle.dump(Homeo_dist_list, picklefile)

In [9]:
#with open('corpus_averages/Mayo_mean.pkl', 'r') as picklefile:
#    Mayo_mean = picklefile
Mayo_mean = pickle.load(open('corpus_averages/Mayo_mean.pkl', 'r'))

In [11]:
Mayo_mean

36489

In [12]:
Homeo_mean = pickle.load(open('corpus_averages/Homeo_mean.pkl', 'r'))
Homeo_mean

65110